In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *

In [0]:
data = [
    (1, "Manish",   26,  75000,  "INDIA",  "m"),
    (2, "Nikita",   23, 100000,  "USA",    "f"),
    (3, "Pritam",   22, 150000,  "INDIA",  "m"),
    (4, "Prantosh", 17, 200000,  "JAPAN",  "m"),
    (5, "Vikash",   31, 300000,  "USA",    "m"),
    (6, "Rahul",    55, 300000,  "INDIA",  "m"),
    (7, "Raju",     67, 540000,  "USA",    "m"),
    (8, "Praveen",  28,  70000,  "JAPAN",  "m"),
    (9, "Dev",      32, 150000,  "JAPAN",  "m"),
    (10,"Sherin",   16,  25000,  "RUSSIA", "f"),
    (11,"Ragu",     12,  35000,  "INDIA",  "f"),
    (12,"Sweta",    43, 200000,  "INDIA",  "f"),
    (13,"Raushan",  48, 650000,  "USA",    "m"),
    (14,"Mukesh",   36,  95000,  "RUSSIA", "m"),
    (15,"Prakash",  52, 750000,  "INDIA",  "m")
]

mySchema = "emp_id int, name string, age int, salary int, country string, gender string"

df = spark.createDataFrame(data, mySchema)

df.show()

In [0]:
df.write.format("csv")\
    .option("header","true")\
    .mode("overwrite")\
    .option("path","/Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/")\
    .save()

As Our Data Frame have multiple partitions so , it is save in multiple CSV files

In [0]:
%fs
ls /Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/

Lets save Whole data/dataframe as only 2 partitions and save them as 2 CSV Files

In [0]:
df.repartition(2).write.format("csv")\
  .option("header","true")\
  .mode("overwrite")\
  .option("path","/Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/")\
  .save()

  # OR

  # df.repartition(2).write.format("csv") \
  #   .option("header", "true") \
  #   .mode("overwrite") \
  #   .save("/Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/")

Now Only data/dataframe got saved into only 2 files (part-00000 & part-00000 )

In [0]:
%fs
ls /Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/

Similiarly We can use other MODES like :
1. append
2. overwrite
3. errorIfExists
4. ignore

### PartitionBy and BucketBy

In [0]:
df.show()

**partitionBy()** : Now Lets save Data in Seprate Folder According To Country

In [0]:
df.write.format("csv")\
  .option("header","true")\
  .mode("overwrite")\
  .partitionBy("country")\
  .option("path","/Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/partitionBy/")\
    .save()

In [0]:
%fs
ls /Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/partitionBy/

We can See data got stored in seprate country folders

Now Lets Try To save data only in 2 partitions for Each Country in Different Country Folder (repatition and partitionBy together)

In [0]:
df.repartition(2).write.format("csv")\
  .option("header","true")\
  .mode("overwrite")\
  .partitionBy("country")\
  .option("path","/Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/partitionBy/")\
    .save()

Lets Partition Data According to COUNTRY AND THEN according to GENDER

In [0]:
df.repartition(2).write.format("csv")\
  .option("header","true")\
  .mode("overwrite")\
  .partitionBy("country","gender")\
  .option("path","/Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/partitionBy/")\
    .save()

In [0]:
%fs
ls dbfs:/Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/partitionBy/country=INDIA/

Above We can See inside each Country Folder we have 2 gender Folder (gender=f, gender=m)

#### bucketBy(no_of_buckets,"col_name") : Don't support CSV format, only parquet and ORC
- .saveAsTable(), create MANAGED TABLE (managed by unity catalog), so we cannot provide """option("path","/filepaath")""" becoz it is way of creating EXTERNAL TABLE
- when we want to segrigate data on A COLUMN, but that column have HIGH CARDENALITY , we use bucketby()
- let try to bucket the data/dataframe on EMP_ID column in 3 buckets
- You generally CANNOT use bucketBy() in Databricks when Unity Catalog is enabled.
But there is one narrow scenario where bucketBy() still works, Using Hive Metastore (legacy)

In [0]:
df.write.format("parquet")\
  .option("header","true")\
  .mode("overwrite")\
  .option("path","/Volumes/workspace/default/rajatlearningdata/PySpark_data/outputs/bucketBy/")\
  .bucketBy(3,"emp_id")\
  .saveAsTable("butcketBy_id_table")

Above Code can work If we use HIVE METASTORE

In [0]:
df.write \
  .format("parquet") \
  .bucketBy(3, "emp_id") \
  .mode("overwrite") \
  .saveAsTable("bucketby_id_table")

Above Error Explain :

In Databricks Unity Catalog (UC):

✅ Managed tables → ONLY Delta format

❌ Parquet / CSV / ORC not allowed for managed tables

saveAsTable() creates a managed table by default

👉 You are asking UC to create a managed Parquet table, which is not permitted.

In [0]:
df.write \
  .format("delta") \
  .bucketBy(3, "emp_id") \
  .mode("overwrite") \
  .saveAsTable("bucketby_id_table")